In [3]:
import numpy as np
import cv2 

In [14]:
# ShiTomasi corner detection
corner_track_par = dict(maxCorners=15,qualityLevel=0.3,
                        minDistance=7, blockSize=7)

In [13]:
# Parameters for lucas kanade optical flow
lk_params = dict(winSize=(200,200),maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03))

In [22]:
# Capture the video
cap = cv2.VideoCapture(0)

# We need the first frame of the stream
ret, prev_frame = cap.read()

# aplly gray scale to the first frame
prev_gray =cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY)

# previous point to track, grabbing the corners
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask=None, **corner_track_par)

# Create a matching mask of the previous frame for drawing on later
mask =np.zeros_like(prev_frame)

while True:

    # Now we need a Current frame
    ret, frame = cap.read()

    # apply gray scale to current frame
    frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    # Calculate the Optical Flow on the Gray Scale Frame !!!
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPts, None,**lk_params)

    good_new = nextPts[status==1]
    good_prev = prevPts[status==1]

    # Use ravel to get points to draw lines and circles
    for i, (new,prev) in enumerate(zip(good_new,good_prev)):

        x_new, y_new = new.ravel()
        x_prev, y_prev = prev.ravel()

        #Draw lines using eprevious frames
        mask = cv2.line(mask,(x_new,y_new),(x_prev,y_prev),(0,255,0),3)

        #Draw red circles using the mask created from the current frame
        frame = cv2.circle(frame, (x_new,y_new),8,(255,0,0),-1)

    img = cv2.add(frame,mask)
    cv2.imshow('traking',img)

    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break

    # Update the previous frame and previous points
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()